<a href="https://colab.research.google.com/github/aworklab/everydayPython/blob/main/ex_20220722.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오늘의 목표

* customer analysis
* iterator

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from datetime import datetime, date, time, timezone, timedelta
import os 

from functools import reduce
from collections import ChainMap, defaultdict, Counter

# customer analysis

In [2]:
os.listdir()

['.config',
 'item_categories.csv',
 'online_retail_II.xlsx',
 'items.csv',
 'sample_submission.csv',
 'rfm_test.csv',
 'retail_data_transactions.csv',
 'retail_data_response.csv',
 '.Rhistory',
 'listings.csv',
 'KPMG_VI_New_raw_data_update_final.xlsx',
 'shops.csv',
 'test.csv',
 'sales_train.csv',
 'ecommerce-data.csv',
 'sample_data']

In [5]:
df = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name = 'Transactions')
df.head()

,Note: The data and information in this document is reflective of a hypothetical situation and client. This document is to be used for KPMG Virtual Internship purposes only.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
1,1,2,2950,2017-02-25 00:00:00,False,Approved,Solex,Standard,medium,medium,71.49,53.62,41245
2,2,3,3120,2017-05-21 00:00:00,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701
3,3,37,402,2017-10-16 00:00:00,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361
4,4,88,3135,2017-08-31 00:00:00,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.1,36145


In [6]:
df.columns = df.iloc[0]
df = df.drop([0], axis = 0)
df.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
1,1,2,2950,2017-02-25 00:00:00,False,Approved,Solex,Standard,medium,medium,71.49,53.62,41245
2,2,3,3120,2017-05-21 00:00:00,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701
3,3,37,402,2017-10-16 00:00:00,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361
4,4,88,3135,2017-08-31 00:00:00,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.1,36145
5,5,78,787,2017-10-01 00:00:00,True,Approved,Giant Bicycles,Standard,medium,large,1765.3,709.48,42226


In [8]:
df['date'] = df.transaction_date.to_numpy().astype('datetime64[D]')
df['month'] = df['date'].to_numpy().astype('datetime64[M]')
df.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,date,month
1,1,2,2950,2017-02-25 00:00:00,False,Approved,Solex,Standard,medium,medium,71.49,53.62,41245,2017-02-25,2017-02-01
2,2,3,3120,2017-05-21 00:00:00,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701,2017-05-21,2017-05-01
3,3,37,402,2017-10-16 00:00:00,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361,2017-10-16,2017-10-01
4,4,88,3135,2017-08-31 00:00:00,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.1,36145,2017-08-31,2017-08-01
5,5,78,787,2017-10-01 00:00:00,True,Approved,Giant Bicycles,Standard,medium,large,1765.3,709.48,42226,2017-10-01,2017-10-01


In [9]:
df = df[(df.order_status == 'Approved')]
df.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,date,month
1,1,2,2950,2017-02-25 00:00:00,False,Approved,Solex,Standard,medium,medium,71.49,53.62,41245,2017-02-25,2017-02-01
2,2,3,3120,2017-05-21 00:00:00,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701,2017-05-21,2017-05-01
3,3,37,402,2017-10-16 00:00:00,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361,2017-10-16,2017-10-01
4,4,88,3135,2017-08-31 00:00:00,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.1,36145,2017-08-31,2017-08-01
5,5,78,787,2017-10-01 00:00:00,True,Approved,Giant Bicycles,Standard,medium,large,1765.3,709.48,42226,2017-10-01,2017-10-01


In [13]:
df = df[['customer_id',  'date', 'month', 'standard_cost']].rename(columns = {'customer_id':'id','standard_cost':'rev'})
df = df.groupby(['id','date','month'])['rev'].sum().reset_index()
df.head()

,id,date,month,rev
0,1,2017-01-05,2017-01-01,270.3
1,1,2017-02-21,2017-02-01,53.62
2,1,2017-03-27,2017-03-01,612.88
3,1,2017-03-29,2017-03-01,764.96
4,1,2017-04-06,2017-04-01,826.51


In [17]:
info_m = df.groupby(['id','month'])['rev'].sum().reset_index().rename(columns = {'rev':'money'})
info_m = pd.pivot_table(info_m, index = 'id', columns = 'month', values = 'money').reset_index()
info_m = pd.melt(info_m, id_vars = 'id', var_name = 'month', value_name = 'money')
info_m = info_m.groupby(['id']).head(3)
info_m = info_m.groupby(['id'])['money'].sum().reset_index()
info_m.head()

,id,money
0,1,1701.76
1,2,0.00
2,3,1856.36
3,4,0.00
4,5,612.88


In [18]:
info_f = df.groupby(['id','month'])['rev'].count().reset_index().rename(columns = {'rev':'freq'})
info_f = pd.pivot_table(info_f, index = 'id', columns = 'month', values = 'freq').reset_index()
info_f = pd.melt(info_f, id_vars = 'id', var_name = 'month', value_name = 'freq')
info_f = info_f.groupby(['id']).head(3)
info_f = info_f.groupby(['id'])['freq'].sum().reset_index()
info_f.head()

,id,freq
0,1,4.0
1,2,0.0
2,3,2.0
3,4,0.0
4,5,1.0


In [19]:
df['date'].max()

Timestamp('2017-12-30 00:00:00')

In [20]:
date_now = datetime(2017,12,31)

In [22]:
info_r = df.groupby('id').agg({'date': lambda x : (date_now - x.max()).days}).reset_index().rename(columns = {'date':'recency'})
info_r.head()

,id,recency
0,1,8
1,2,129
2,3,103
3,4,196
4,5,17


In [31]:
info = pd.merge(pd.merge(info_r, info_f, 'left', 'id'), info_m, 'left', 'id')
info.head()

,id,recency,freq,money
0,1,8,4.0,1701.76
1,2,129,0.0,0.00
2,3,103,2.0,1856.36
3,4,196,0.0,0.00
4,5,17,1.0,612.88


In [38]:
info = info.assign(r = pd.qcut(info.recency, 5, list('54321')),
            f = pd.qcut(info.freq.rank(method = 'first'), 5, list('12345')),
            m = pd.qcut(info.money.rank(method = 'first'), 5, list('12345'))
            )
info.head()

,id,recency,freq,money,r,f,m
0,1,8,4.0,1701.76,5,5,5
1,2,129,0.0,0.00,1,1,1
2,3,103,2.0,1856.36,1,3,5
3,4,196,0.0,0.00,1,1,1
4,5,17,1.0,612.88,4,2,3


In [39]:
info.assign(r = info.r.astype(str) + info.f.astype(str))

,id,recency,freq,money,r,f,m
0,1,8,4.0,1701.760000,55,5,5
1,2,129,0.0,0.000000,11,1,1
2,3,103,2.0,1856.360000,13,3,5
3,4,196,0.0,0.000000,11,1,1
4,5,17,1.0,612.880000,42,2,3
...,...,...,...,...,...,...,...
3488,3497,53,0.0,0.000000,32,2,2
3489,3498,128,3.0,508.550000,15,5,3
3490,3499,52,3.0,667.620000,35,5,3
3491,3500,145,4.0,1915.750000,15,5,5


# iterator

In [40]:
mat = [list('ABC'), list('DEF'), list('HIG')]
mat

[['A', 'B', 'C'], ['D', 'E', 'F'], ['H', 'I', 'G']]

In [41]:
for r, rows in enumerate(mat) :
  for c, letter in enumerate(mat) :
    print(f'({r}, {c}) : {letter}')

(0, 0) : ['A', 'B', 'C']
(0, 1) : ['D', 'E', 'F']
(0, 2) : ['H', 'I', 'G']
(1, 0) : ['A', 'B', 'C']
(1, 1) : ['D', 'E', 'F']
(1, 2) : ['H', 'I', 'G']
(2, 0) : ['A', 'B', 'C']
(2, 1) : ['D', 'E', 'F']
(2, 2) : ['H', 'I', 'G']


In [42]:
users = [{'mail': 'gregorythomas@gmail.com', 'name': 'Brett Holland', 'sex': 'M', 'age': 73},
 {'mail': 'hintoncynthia@hotmail.com', 'name': 'Madison Martinez', 'sex': 'F', 'age': 29},
 {'mail': 'wwagner@gmail.com', 'name': 'Michael Jenkins', 'sex': 'M', 'age': 51},
 {'mail': 'daniel79@gmail.com', 'name': 'Karen Rodriguez', 'sex': 'F', 'age': 32},
 {'mail': 'ujackson@gmail.com', 'name': 'Amber Rhodes', 'sex': 'F', 'age': 42}]
users

[{'age': 73,
  'mail': 'gregorythomas@gmail.com',
  'name': 'Brett Holland',
  'sex': 'M'},
 {'age': 29,
  'mail': 'hintoncynthia@hotmail.com',
  'name': 'Madison Martinez',
  'sex': 'F'},
 {'age': 51,
  'mail': 'wwagner@gmail.com',
  'name': 'Michael Jenkins',
  'sex': 'M'},
 {'age': 32,
  'mail': 'daniel79@gmail.com',
  'name': 'Karen Rodriguez',
  'sex': 'F'},
 {'age': 42, 'mail': 'ujackson@gmail.com', 'name': 'Amber Rhodes', 'sex': 'F'}]

In [43]:
for keys in list(users[0].keys()) :
  if keys == list(users[0].keys())[0] :
    res = eval(f'reduce(lambda acc, cur : acc + [cur["{keys}"]], users, [])')
  else :
    res = np.c_[res, eval(f'reduce(lambda acc, cur : acc + [cur["{keys}"]], users, [])')]

res

array([['gregorythomas@gmail.com', 'Brett Holland', 'M', '73'],
       ['hintoncynthia@hotmail.com', 'Madison Martinez', 'F', '29'],
       ['wwagner@gmail.com', 'Michael Jenkins', 'M', '51'],
       ['daniel79@gmail.com', 'Karen Rodriguez', 'F', '32'],
       ['ujackson@gmail.com', 'Amber Rhodes', 'F', '42']], dtype='<U25')

In [44]:
portfolio = [
    ('GOOG', 100, 490.1),
    ('IBM', 50, 91.1),
    ('CAT', 150, 83.44),
    ('IBM', 100, 45.23),
    ('GOOG', 75, 572.45),
    ('AA', 50, 23.15)
]
portfolio

[('GOOG', 100, 490.1),
 ('IBM', 50, 91.1),
 ('CAT', 150, 83.44),
 ('IBM', 100, 45.23),
 ('GOOG', 75, 572.45),
 ('AA', 50, 23.15)]

In [45]:
a = Counter()
for com, share, price in portfolio : 
  a[com] =+ share
a

Counter({'AA': 50, 'CAT': 150, 'GOOG': 75, 'IBM': 100})

In [46]:
b = defaultdict(list)
for com, share, price in portfolio :
  b[com].append((share, price))
b

defaultdict(list,
            {'AA': [(50, 23.15)],
             'CAT': [(150, 83.44)],
             'GOOG': [(100, 490.1), (75, 572.45)],
             'IBM': [(50, 91.1), (100, 45.23)]})